<a href="https://colab.research.google.com/github/arizzi/NNTutorial/blob/master/CNN_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tutorial on CNN
We try to build a DNN that recognize if an image contains a rectangle or a circle

## Import useful stuff

In [0]:
from keras.layers import Input,Dense,Dropout,Conv2D,MaxPooling2D,Flatten
from keras.models import Model
import numpy as np
from math import *
from matplotlib import pyplot as plt 

## Lets generate some data

We now generate ourself some images with a circle or a rectangle, of random color, in a random position

### Exercise
1. Try adding more classes such e.g. Lines or Ellipses

2. Try adding some random noise in the image background

In [0]:
import cv2

def randomColor():
  return (int(np.random.rand()*256),int(np.random.rand()*256),int(np.random.rand()*256))

def drawCircle(c,x,y,r):
  img = np.zeros((64,64,3), np.uint8)
  cv2.circle(img,(x,y),r,c, -1)
  return img

def genCircle():
  return drawCircle(randomColor(),int(np.random.rand()*30)+10,int(np.random.rand()*30)+10,
                    int(np.random.rand()*12)+5)

def drawRectangle(c,x,y,w,h):
  img = np.zeros((64,64,3), np.uint8)
  cv2.rectangle(img,(x,y),((x+w),(y+h)), c, -1)
  return img

def genRectangle():
  return drawRectangle(randomColor(),int(np.random.rand()*30)+10,int(np.random.rand()*30)+10,
                       int(np.random.rand()*12)+5,int(np.random.rand()*12)+5)


nsamples=500
rects=np.stack([genRectangle() for x in range(nsamples)])
circs=np.stack([genCircle() for x in range(nsamples)])


Let's show one of the generated image

In [0]:
print(rects.shape)
plt.imshow(rects[1])

We concatenate zeroes and ones as labels and the shuffle with the same permutation both the data and the labels.
## Exercise
3. If we have more categories (let say N) we should use a categorical label that is a vector of length N with 1 on the category(/ies) the image belong to and 0 in the others. Try to build a categorical label for two categories (e.g. using numpy "tile" function to repeat the same raw multiple times )
4. Expand the categorical label to  Ellispes or  Lines  and possibly also non exclusive categories such has 2D vs 1D objects

In [0]:
labels=np.concatenate((np.zeros(rects.shape[0]),np.ones(circs.shape[0])))
data=np.concatenate((rects,circs))
permutation = np.random.permutation(data.shape[0])
data=data[permutation]
labels=labels[permutation]

# Let's build a CNN 

Now we build our first CNN. We have some Conv layers interleaved with MaxPooling

Finally we flatten the output of the convolutional stack and appply a Dense FF

### MaxPooling
![alt text](https://computersciencewiki.org/images/8/8a/MaxpoolSample2.png)

### Exercise
5. Try adding/removing convolutional layers, change the kernel size.
6. Try changing the model to categorical labels, change loss function from binary_crossentropy to categorical_crossentropy, and use softmax activation instead of sigmoid


In [0]:
inputs=Input(shape=(64,64,3,))
hidden=  Conv2D(6,(3,3), activation='relu')(inputs)
hidden=  Conv2D(3,(3,3), activation='relu')(hidden)
hidden= MaxPooling2D((8,8))(hidden)
hidden=  Conv2D(3,(3,3), activation='relu')(hidden)
hidden= MaxPooling2D((4,4))(hidden)
hidden= Flatten()(hidden)
hidden=  Dense(10, activation='relu')(hidden)
hidden=  Dense(10, activation='relu')(hidden)
hidden=  Dense(10, activation='relu')(hidden)
outputs = Dense(1, activation='sigmoid')(hidden)
model = Model(input=inputs, output=outputs)
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

model.summary()

And now let's fit it to our data.
The sample is automatically split in two so that 50% of it is used for validation and the other half for training


In [0]:
history=model.fit(data,labels,validation_split=0.5,epochs=25)

*history* contains information about the training.  We can now now show the loss vs epoch for both validation and training samples.




In [0]:
print(history.history.keys())
plt.plot(history.history["val_loss"])
plt.plot(history.history["loss"])
plt.show()
plt.plot(history.history["val_acc"])
plt.plot(history.history["acc"])
plt.show()


In [0]:
print(model.predict(np.expand_dims(genRectangle(),axis=0) ))
print(model.predict(np.expand_dims(genCircle(),axis=0) ))


Let's try to mix an image with circles and rectangles and see how the network would evaluate it 

In [0]:
im=genCircle()
print("Looks like a", ("circle" if model.predict(np.expand_dims(im,axis=0) )> 0.5 else "rectangle"))
plt.imshow(im)

The following code can be used to visualize what features the first conv layer is looking at

In [0]:
# retrieve weights from the second hidden layer
filters, biases = model.layers[1].get_weights()
# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)
# plot first few filters
n_filters, ix = 6, 1
for i in range(n_filters):
	# get the filter
	f = filters[:, :, :, i]
	# plot each channel separately
	for j in range(3):
		# specify subplot and turn of axis
		ax = plt.subplot(n_filters, 3, ix)
		ax.set_xticks([])
		ax.set_yticks([])
		# plot filter channel in grayscale
		plt.imshow(f[:, :, j], cmap='gray')
		ix += 1
# show the figure
plt.show()